In [1]:
from EMAN2 import *
import numpy as np

In [2]:
rf=EMData("/home2/stevel/test3.hdf",2) 
rf.process_inplace("normalize.edgemean")

In [5]:
im=EMData("/home2/stevel/test3.hdf",1)
im.process_inplace("mask.fft.peak",{"removepeaks":1,"thresh_sigma":1.0,"to_mean":0})
im.process_inplace("filter.lowpass.tophat",{"cutoff_abs":0.5})
im.process_inplace("mask.soft",{"outer_radius":250.0,"width":3.0})

#im.process_inplace("filter.lowpass.gauss",{"cutoff_freq":0.05})
#im.process_inplace("filter.highpass.gauss",{"cutoff_pixels":5})
e3display([im,rf],"image","target")

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xfa\x00\x00\x01\xfa\x08\x06\x00\x00\x00\xef\x15\…

In [60]:
def fmin(a): return a.index(min(a))

In [6]:
# compute CCF for locally masked regions of target image to try and align each piece independently
out=[]
mskd=[]
xs=[]
ys=[]
xvs=[]
yvs=[]
vs=[]
for x in range(32,512,32):
    for y in range(32,512,32):
        imm=im.process("mask.gaussian",{"dx":x-256,"dy":y-256,"outer_radius":80})
        imm.mult(10.0)
        imm.add(im)
        imm.process_inplace("normalize.unitlen")
#        rf2=rf.process("mask.sharp",{"dx":x-256,"dy":y-256,"outer_radius":80})
        rf2=rf.copy()
        rf2.process_inplace("normalize.unitlen")
        ccf=imm.calc_ccf(rf2)
#        ccf=imm.calc_ccf(im)

        ccf.process_inplace("xform.phaseorigin.tocenter")
#        ccf.process_inplace("mask.sharp",{"outer_radius":40})
        a,b,c=ccf.calc_max_location()
        c=ccf[a,b]
        if c<0.1: continue
        xs.append(x)
        ys.append(y)
        xvs.append(256-a)
        yvs.append(256-b)
        vs.append(c)
        imm2=imm.copy()
        imm2.translate(256-a,256-b,0)
        mskd.append(imm)
        mskd.append(imm2)
        mskd.append(rf2)
        out.append(ccf)

xs=np.array(xs)
ys=np.array(ys)
xvs=np.array(xvs)
yvs=np.array(yvs)
vs=np.array(vs)
e3display((xs,ys,xvs,yvs,vs),"plot3d","allccfs","allccfs")
xvs+=xs
yvs+=ys
e3display(out,"image","ccfs","ccfs")
e3display(mskd,"image","masked","masked")
e3display((xs,ys,xvs,yvs,vs),"image","target")


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xfa\x00\x00\x01\xfa\x08\x06\x00\x00\x00\xef\x15\…

In [7]:
e3display([rf,mskd[83]],"image","ref","ref")

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xfa\x00\x00\x01\xfa\x08\x06\x00\x00\x00\xef\x15\…

In [8]:
#exhaustive local translation search using multiple metrics with 3-D plot
seq=[]
xs=[]
ys=[]
cdot=[]
cfrc=[]
clod=[]
copts=[]
for y in range(-5,30,1):
    for x in range(-5,30,1):
        imm=im.copy()
        imm.translate(x,y,0)
        xs.append(x)
        ys.append(y)
        cdot.append(imm.cmp("ccc",rf))
        cfrc.append(imm.cmp("frc",rf,{"minres":100,"maxres":20}))
        clod.append(imm.cmp("lod",rf))
        copts.append(imm.cmp("optsub",rf,{"minres":100,"maxres":20,"zeromask":1}))
        seq.append(imm)
        seq.append(rf)

e3display(seq,"image","masked","masked")
e3display((xs,ys,cdot,cfrc,clod,copts),"plot3d","allccfs","cmps")



Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02z\x00\x00\x01\xda\x08\x02\x00\x00\x00\x18<s(\x00\…

In [55]:
tst=cdot
i=tst.index(min(tst))
print(i,xs[i],ys[i],i*2)

384 29 5 768


In [103]:
#exhaustive local translation search using multiple metrics with 3-D plot
seq=[]
xs=[]
ys=[]
cfrc1=[]
cfrc2=[]
cfrc3=[]
cfrc4=[]
rf2=rf.process("mask.gaussian",{"outer_radius":96})
rf2.process_inplace("normalize.edgemean")
for y in range(-30,30,2):
    for x in range(-30,30,2):
        imm=im.copy()
        imm.translate(x,y,0)
        imm.process_inplace("mask.gaussian",{"outer_radius":96})
        imm.process_inplace("normalize.edgemean")
        xs.append(x)
        ys.append(y)
        cfrc1.append(imm.cmp("frc",rf2,{"minres":350,"maxres":80}))
        cfrc2.append(imm.cmp("frc",rf2,{"minres":350,"maxres":60}))
        cfrc3.append(imm.cmp("frc",rf2,{"minres":350,"maxres":40}))
        cfrc4.append(imm.cmp("ccc",rf2))
        seq.append(imm)
        seq.append(rf)

e3display(seq,"image","masked","masked")
e3display((xs,ys,cfrc1,cfrc2,cfrc3,cfrc4),"plot3d","allccfs","frc")
print(fmin(cfrc1),fmin(cfrc2),fmin(cfrc3),fmin(cfrc4))
print(fmin(cfrc1)*2,fmin(cfrc2)*2,fmin(cfrc3)*2,fmin(cfrc4)*2)
print(xs[fmin(cfrc1)],xs[fmin(cfrc2)],xs[fmin(cfrc3)],xs[fmin(cfrc4)])
print(ys[fmin(cfrc1)],ys[fmin(cfrc2)],ys[fmin(cfrc3)],ys[fmin(cfrc4)])



541 540 512 880
1082 1080 1024 1760
-28 -30 -26 -10
6 6 4 28


In [9]:
#exhaustive local translation search using multiple metrics with 3-D plot
seq=[]
xs=[]
ys=[]
cfrc1=[]
cfrc2=[]
cfrc3=[]
cfrc4=[]
rf2=rf.process("mask.soft",{"outer_radius":128})
rf2.process_inplace("normalize.edgemean")
for y in range(-5,40,2):
    for x in range(-5,40,2):
        imm=im.copy()
        imm.translate(x,y,0)
        imm.process_inplace("mask.soft",{"outer_radius":128})
        imm.process_inplace("normalize.edgemean")
        xs.append(x)
        ys.append(y)
        cfrc1.append(imm.cmp("optsub",rf2,{"minres":250,"maxres":60,"zeromask":1}))
        cfrc2.append(imm.cmp("optsub",rf2,{"minres":230,"maxres":60,"zeromask":1}))
        cfrc3.append(imm.cmp("optsub",rf2,{"minres":210,"maxres":60,"zeromask":1}))
        cfrc4.append(imm.cmp("optsub",rf2,{"minres":190,"maxres":60,"zeromask":1}))
        seq.append(imm)
        seq.append(rf2)

e3display(seq,"image","masked","masked")
e3display((xs,ys,cfrc1,cfrc2,cfrc3,cfrc4),"plot3d","allccfs","optsub")
print(fmin(cfrc1),fmin(cfrc2),fmin(cfrc3),fmin(cfrc4))
print(fmin(cfrc1)*2,fmin(cfrc2)*2,fmin(cfrc3)*2,fmin(cfrc4)*2)
print(xs[fmin(cfrc1)],xs[fmin(cfrc2)],xs[fmin(cfrc3)],xs[fmin(cfrc4)])
print(ys[fmin(cfrc1)],ys[fmin(cfrc2)],ys[fmin(cfrc3)],ys[fmin(cfrc4)])

NameError: name 'fmin' is not defined

In [74]:
xs[382//2],ys[382//2]

(9, 11)

In [113]:
v=EMData("/home2/stevel/test2dd_p.hdf",144)
vf=v.do_fft()

In [107]:
e3display(v,"image","test","test")

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xfa\x00\x00\x01\xfa\x08\x02\x00\x00\x00`w\x19\x1…

In [149]:
amp=np.array(vf.calc_radial_dist(128,0,2,0))
inten=np.array(vf.calc_radial_dist(128,0,2,1))
az=np.array(vf.calc_az_dist(180,-90,1,20,40))
sig=np.sqrt((inten-amp**2.0))
e3display(amp,"plot2d","az","amp")
e3display(sig,"plot2d","az","sig")
e3display(az,"plot2d","az","az1")

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xed\x00\x00\x01\xda\x08\x02\x00\x00\x00\xfa\xc5?…

In [131]:
ad.mean(),ad.std(),np.median(ad)


(584.2025989956326, 458.1358177714262, 486.26832580566406)

In [136]:
help(EMData.calc_radial_dist)

Help on built-in function calc_radial_dist:

calc_radial_dist(...)
    calc_radial_dist( (object)arg1, (object)n, (object)x0, (object)dx, (object)inten) -> object :
        calculates radial distribution. works for real and imaginary images.
        inten=0->mean amp, 1->mean inten (amp^2), 2->min, 3->max, 4->sigma. Note that the complex
        origin is at (0,0), with periodic boundaries. Note that the inten option is NOT
        equivalent to returning amplitude and squaring the result.
         
        n - number of points.
        x0 - starting point x coordinate.
        dx - step of x.
        inten returns intensity (amp^2) rather than amplitude if set
         
        return The radial distribution in an array.
    
        C++ signature :
            std::vector<float, std::allocator<float> > calc_radial_dist(EMAN::EMData {lvalue},int,float,float,int)
    
    calc_radial_dist( (object)arg1, (object)n, (object)x0, (object)dx, (object)nwedge, (object)offset, (object)inten) -